## Objectives
- Learn how to create a graphical user interface (GUI) for the WeatherVista project.
- Understand the basics of Jupyter Widgets building notebook applications.
- Explore using Jupyter Widgets for interactive elements in Jupyter notebooks.
  
## Setting Up the Environment

Begin by installing the ipywidgets library and enabling the widgets extension in Jupyter Notebooks to ensure a seamless interactive experience. This step sets the foundation for integrating dynamic controls into your notebooks.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
import weather_plots as wp 
import matplotlib.patches as patches
from fetch_data import fetch_weather_data
from process_data import process_weather_data
from utils import determine_weather

## Fetching Weather Data
Utilize APIs such as OpenWeatherMap to fetch current weather data. This step involves making HTTP requests and processing the received JSON data to extract relevant weather information.

In [ ]:
api_key = 'd0de46031ad7410d0c72c6063690e1d0'  # Replace with your OpenWeatherMap API key
city = 'Sydney'  # Default city for demonstration
current_data, forecast_data = fetch_weather_data(api_key, city)
current_df, hourly_df, daily_df = process_weather_data(current_data, forecast_data)

## Determining Weather Conditions
Analyze the fetched data to determine specific weather conditions like temperature, humidity, and wind speed. This processing helps in displaying meaningful information and creating visualizations.

In [ ]:
daily_df['Weather'] = daily_df.apply(determine_weather, axis=1, args=(25, 15, 80))

### 5. Creating Dropdown Widget
Create a dropdown widget using ipywidgets to allow users to select different cities or weather parameters. This widget will enable dynamic interaction and updates based on user input.

In [ ]:
city_dropdown = widgets.Dropdown(
    options=['Sydney', 'New York', 'London', 'Beijing'],
    value='Sydney',
    description='City:'
)
display(city_dropdown)

### 6. Creating Update Button
Add an update button to trigger the fetching and processing of weather data based on the selected dropdown options. This button will help refresh the displayed information without re-running the entire notebook.

In [ ]:
update_button = widgets.Button(
    description='Update Weather',
    button_style='success'
)
display(update_button)

### 7. Output Area for Plots

Designate an output area using ipywidgets where the weather plots will be displayed. This area will dynamically update as the user interacts with the dropdown and update button.

In [ ]:
output = widgets.Output()
display(output)

### 8. Updating Weather Data and Plots

Implement functionality to update the weather data and corresponding plots when the user changes the dropdown selection or clicks the update button. This ensures the displayed data is always current and relevant.

In [ ]:
def update_weather(b):
    city = city_dropdown.value
    current_data, forecast_data = fetch_weather_data(api_key, city)
    current_df, hourly_df, daily_df = process_weather_data(current_data, forecast_data)
    daily_df['Weather'] = daily_df.apply(determine_weather, axis=1, args=(25, 15, 80))
    
    with output:
        output.clear_output()
        fig, axs = plt.subplots(1, 3, figsize=(15, 4))
        fig.tight_layout(pad=5.0)

        # Generate plots
        wp.create_forecast_card(axs[0], daily_df.iloc[0]['Date'], daily_df.iloc[0]['Max Temperature (C)'], daily_df.iloc[0]['Min Temperature (C)'], daily_df.iloc[0]['Weather'])
        wp.plot_scatter(hourly_df, 'Temperature (C)', 'Wind Speed (m/s)', axs[1], title="Wind Speed vs Temperature")
        wp.plot_min_max(daily_df, 'Date', 'Min Temperature (C)', 'Max Temperature (C)', axs[2], title="Daily Min/Max Temperature")

        plt.show()

## Add events and display initial dashboard

Bind the created widgets to events and display the initial dashboard layout. This setup will prepare the interface for user interactions, showing the initial state of the weather data and plots.

In [ ]:
update_button.on_click(update_weather)
display(city_dropdown, update_button, output)
update_weather(None)

## Combining Everything

Integrate all components into a cohesive interactive dashboard that updates weather data and visualizations in real-time based on user input. This final step brings together the dropdown, update button, and output area to create a functional and dynamic weather dashboard.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
import weather_plots as wp  # Assuming this contains your plotting functions
import matplotlib.patches as patches
from fetch_data import fetch_weather_data
from process_data import process_weather_data

def determine_weather(row, sunny_threshold, rainy_min_temp_threshold, rainy_humidity_threshold):
    if row['Max Temperature (C)'] > sunny_threshold:
        return 'Sunny'
    elif row['Min Temperature (C)'] < rainy_min_temp_threshold and row['Max Humidity (%)'] > rainy_humidity_threshold:
        return 'Rainy'
    else:
        return 'Partly Cloudy'

# Dropdown for city selection
city_dropdown = widgets.Dropdown(
    options=['Sydney', 'New York', 'London', 'Beijing'],
    value='Sydney',
    description='City:'
)

# Button to fetch and update weather data
update_button = widgets.Button(
    description='Update Weather',
    button_style='success'
)

output = widgets.Output()

# Function to update weather data and plots
def update_weather(b):
    city = city_dropdown.value
    api_key = 'd0de46031ad7410d0c72c6063690e1d0'  # Replace with your OpenWeatherMap API key
    current_data, forecast_data = fetch_weather_data(api_key, city)
    current_df, hourly_df, daily_df = process_weather_data(current_data, forecast_data)
    daily_df['Weather'] = daily_df.apply(determine_weather, axis=1, args=(25, 15, 80))

    with output:
        output.clear_output()
        fig, axs = plt.subplots(1, 3, figsize=(15, 4))
        fig.tight_layout(pad=5.0)

        # Generate plots
        wp.create_forecast_card(axs[0], daily_df.iloc[0]['Date'], daily_df.iloc[0]['Max Temperature (C)'], daily_df.iloc[0]['Min Temperature (C)'], daily_df.iloc[0]['Weather'])
        wp.plot_scatter(hourly_df, 'Temperature (C)', 'Wind Speed (m/s)', axs[1], title="Wind Speed vs Temperature")
        wp.plot_min_max(daily_df, 'Date', 'Min Temperature (C)', 'Max Temperature (C)', axs[2], title="Daily Min/Max Temperature")

        plt.show()

update_button.on_click(update_weather)
display(city_dropdown, update_button, output)
update_weather(None)